# ready 

In [1]:
from handler_demographic import merge_demographic
from handler_GIS_demographic import build_GIS_demographic_data
from handler_GIS_buslines_jerusalem import make_jlm_metro_routes
from handler_rishui import rishui_resolutions, passengers_resolutions, ebitzua_resolutions
from handler_socioeconomic_ranking import calculate_socio_econimoc_lines
from handler_service_areas import create_service_areas_resolutions
from handler_complaints import complaints_resolutions
from general_functions import prepare_data_to_directness_analysis
from config_general import * 

## use arcgis pro kernel

In [2]:
demographic_table=merge_demographic() 
build_GIS_demographic_data()
bus_lines_jlm=make_jlm_metro_routes()
risui_res12, risui_res3=rishui_resolutions(bus_lines_jlm)
res_12, res3=calculate_socio_econimoc_lines(bus_lines_jlm)

service_areas_res12, service_areas_res3=create_service_areas_resolutions(risui_res12)

passengers_res12, passengers_res3=passengers_resolutions()

ebitzua_res1, ebitzua_res2, ebitzua_res3=ebitzua_resolutions()

complaints_res1,complaints_res2, complaints_res3=complaints_resolutions()

prepare_data_to_directness_analysis()

statistical areas with demographic date is printed
rishui data res12 is printed
rishui data res3 is printed
sosio economic ranking printed
service areas are printed
passengers numbers per line are printed
e-bitua data printed
scomplaints data printed
pre directness analysis complete


## change kernel to ox, and run the cell. when done,change back to arcgis pro kernel

In [2]:
# from handler_directness import create_directness_measurement
# create_directness_measurement()

## change back to arcgus pro kernel

In [3]:
from handler_merger_export import *
merge_and_export()

ModuleNotFoundError: No module named 'handler_merger_export'

In [ ]:
directness_res12, directness_res3=create_directness()

res1_dfs=[risui_res12[['makat', 'type']],passengers_res12,directness_res12[['route_desc', 'directness']],service_areas_res12]


res2_dfs=[risui_res12[['makat', 'type']],passengers_res12,ebitzua_res2,
          directness_res12[['route_desc', 'directness']],service_areas_res12, complaints_res2]

res3_dfs=[risui_res3[['routeid_direction', 'type']],passengers_res3,ebitzua_res3,
              directness_res3,service_areas_res3, complaints_res3]

outputr_res1=create_res1(res1_dfs)
output_res2=create_res2(res2_dfs)
output_res3=create_res3(res3_dfs)

In [56]:
def create_directness():
    directness=pd.read_csv('outputs/final_directness.csv')
    directness_res12=directness[['route_desc', 'directness']].copy()

    directness[['routeid','Direction','Alternative']] = directness['route_desc'].str.split('-',expand=True)
    directness['routeid_direction']=directness['routeid'].astype(str)+'-'+directness['Direction'].astype(str)
    directness_res3=directness.groupby('routeid_direction', as_index=False)['directness'].mean()
    return directness_res12, directness_res3

In [66]:

def create_res1(res1_dfs):
    res1=ebitzua_res1.merge(complaints_res1, on=['makat','day_period'])
    day_periods=res1['day_period'].drop_duplicates().to_frame()
    day_periods['tmp'] =1 

    res1_dfs=[risui_res12[['makat', 'type']],passengers_res12,directness_res12[['route_desc', 'directness']],service_areas_res12]
    for i,df in enumerate(res1_dfs):
        if 'makat' not in df.columns:
            df.rename(columns={'route_desc':'makat'}, inplace=True)
        if i==0:
            output_res1=df
        else:
            output_res1=output_res1.merge(df, on ='makat')

    output_res1['tmp'] =1 
    ready_tomerge=output_res1.merge(day_periods, on='tmp', how='left')


    output_res1=ready_tomerge.merge(res1,on=['makat', 'day_period'], how='left')
    output_res1.drop(columns='tmp', inplace=True)
    return output_res1.drop_duplicates()

In [67]:

def create_res2(res2_dfs):
    for i,df in enumerate(res2_dfs):
        if 'makat' not in df.columns:
            df.rename(columns={'route_desc':'makat'}, inplace=True)
        if i==0:
            output_res2=df
        else:
            output_res2=output_res2.merge(df, on ='makat')
    return output_res2.drop_duplicates()

In [68]:

def create_res3(res3_dfs):
    for i,df in enumerate(res3_dfs):
        if i==0:
            output_res3=df
        else:
            output_res3=output_res3.merge(df, on ='routeid_direction')
    return output_res3.drop_duplicates()

In [ ]:
def export_csv(dfs):
    for int, res in enumerate(dfs):
        int=int+1

        name_dict={'eastwest':'East_JLM_lines', 
            'type': 'service_types', 
        'socioeco_2019':'Socioeconomic_ranking', 
        'updated_ultraorthodox_percent':'Ultraorthodox_buses',
        'directness':'Directness_measurements', 
        'settlements': 'Settlements_lines', 
        'west':'West_metropolin_lines', 
        'injlm':'InJerusalem_lines',
        'headway':'Lines_headway', 
        'not_intact_percent':'percent_problematic_trips'
    }
        res.rename(columns=name_dict, inplace=True)

        general_columns=['East_JLM_lines', 'service_types', 
            'Socioeconomic_ranking', 'Ultraorthodox_buses',
            'Directness_measurements', 'Settlements_lines',
            'West_metropolin_lines', 'InJerusalem_lines',
            'percent_problematic_trips', 'number_of_complaints',
            'passengersnumber_thousands']
        if int==1:
            cols=['makat', 'day_period']
        elif int==2:
            cols=['makat']
        else:
            cols=['routeid_direction']
        
        export_columns=cols+general_columns
    
        res[export_columns].to_csv(f'saved_steps/res{int}.csv', index=False)
        print(f'res{int} exported')

In [ ]:
def export_to_excel(dfs):
    for int, res in enumerate(dfs):
        int=int+1

        name_dict={'eastwest':'East_JLM_lines', 
            'type': 'service_types', 
            'passengersnumber':'passengersnumber_thousands',
        'directness':'Directness_measurements', 
        'settlements': 'Settlements_lines', 
        'west':'West_metropolin_lines', 
        'injlm':'InJerusalem_lines',
        'headway':'Lines_headway', 
        'not_intact_percent':'percent_problematic_trips'
    }
        res.rename(columns=name_dict, inplace=True)

        general_columns=['East_JLM_lines', 'service_types', 
            'Directness_measurements', 'Settlements_lines',
            'West_metropolin_lines', 'InJerusalem_lines',
            'percent_problematic_trips', 'number_of_complaints',
            'passengersnumber_thousands']
        if int==1:
            cols=['makat', 'day_period']
        elif int==2:
            cols=['makat']
        else:
            cols=['routeid_direction']
        
        export_columns=cols+general_columns

        res[export_columns].to_csv(f'outputs/res{int}.csv', index=False)
        print(f'res{int} exported')

In [ ]:
def merge_and_export():
    directness_res12, directness_res3=create_directness()

    res1_dfs=[risui_res12[['makat', 'type']],passengers_res12,directness_res12[['route_desc', 'directness']],service_areas_res12]


    res2_dfs=[risui_res12[['makat', 'type']],passengers_res12,ebitzua_res2,
            directness_res12[['route_desc', 'directness']],service_areas_res12, complaints_res2]

    res3_dfs=[risui_res3[['routeid_direction', 'type']],passengers_res3,ebitzua_res3,
                directness_res3,service_areas_res3, complaints_res3]

    outputr_res1=create_res1(res1_dfs)
    output_res2=create_res2(res2_dfs)
    output_res3=create_res3(res3_dfs)

    dfs=[outputr_res1,output_res2,output_res3]
    export_to_excel(dfs)

In [77]:
outputr_res1

,makat,service_types,passengersnumber,Directness_measurements,Settlements_lines,West_metropolin_lines,East_JLM_lines,InJerusalem_lines,day_period,percent_problematic_trips,number_of_complaints
0,12035-2-ב,3,7726,0.41,0,0,0,1,3,0.00,1.0
1,12035-2-ב,3,7726,0.41,0,0,0,1,1,0.00,2.0
2,12035-2-ב,3,7726,0.41,0,0,0,1,2,0.00,1.0
3,14023-3-#,3,10446,0.42,0,0,0,1,3,0.00,1.0
4,14023-3-#,3,10446,0.42,0,0,0,1,1,0.00,2.0
...,...,...,...,...,...,...,...,...,...,...,...
1945,10214-3-2,3,54,0.97,0,0,1,0,1,NaN,NaN
1946,10214-3-2,3,54,0.97,0,0,1,0,2,NaN,NaN
1947,10213-3-#,3,249,0.02,0,0,1,1,3,NaN,NaN
1948,10213-3-#,3,249,0.02,0,0,1,1,1,33.33,2.0


In [ ]:
def excport_to_excel():
    for int, res in enumerate([outputr_res1,output_res2,output_res3]):
        int=int+1

        name_dict={'eastwest':'East_JLM_lines', 
            'type': 'service_types', 
            'passengersnumber':'passengersnumber_thousands',
        'directness':'Directness_measurements', 
        'settlements': 'Settlements_lines', 
        'west':'West_metropolin_lines', 
        'injlm':'InJerusalem_lines',
        'headway':'Lines_headway', 
        'not_intact_percent':'percent_problematic_trips'
    }
        res.rename(columns=name_dict, inplace=True)

        general_columns=['East_JLM_lines', 'service_types', 
            'Directness_measurements', 'Settlements_lines',
            'West_metropolin_lines', 'InJerusalem_lines',
            'percent_problematic_trips', 'number_of_complaints',
            'passengersnumber_thousands']
        if int==1:
            cols=['makat', 'day_period']
        elif int==2:
            cols=['makat']
        else:
            cols=['routeid_direction']
        
        export_columns=cols+general_columns

        res[export_columns].to_csv(f'outputs/res{int}.csv', index=False)
        print(f'res{int} exported')

res1 exported
res2 exported
res3 exported


In [55]:
res1_dfs=[risui_res12[['makat', 'type']],passengers_res12,directness_res12[['route_desc', 'directness']],service_areas_res12]

def create_res1(res1_dfs):
    res1=ebitzua_res1.merge(complaints_res1, on=['makat','day_period'])
    day_periods=res1['day_period'].drop_duplicates().to_frame()
    day_periods['tmp'] =1 

    res1_dfs=[risui_res12[['makat', 'type']],passengers_res12,directness_res12[['route_desc', 'directness']],service_areas_res12]
    for i,df in enumerate(res1_dfs):
        if 'makat' not in df.columns:
            df.rename(columns={'route_desc':'makat'}, inplace=True)
        if i==0:
            output_res1=df
        else:
            output_res1=output_res1.merge(df, on ='makat')

    output_res1['tmp'] =1 
    ready_tomerge=output_res1.merge(day_periods, on='tmp', how='left')


    output_res1=ready_tomerge.merge(res1,on=['makat', 'day_period'], how='left')
    output_res1.drop(columns='tmp', inplace=True)

In [ ]:
res2_dfs=[risui_res12[['makat', 'type']],passengers_res12,ebitzua_res2,directness[['route_desc', 'directness']],service_areas_res12, complaints_res2]
for i,df in enumerate(res2_dfs):
    if 'makat' not in df.columns:
        df.rename(columns={'route_desc':'makat'}, inplace=True)
    if i==0:
        output_res2=df
    else:
        output_res2=output_res2.merge(df, on ='makat')

C:\Users\titib\AppData\Local\Temp\ipykernel_8784\316768176.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'route_desc':'makat'}, inplace=True)


In [ ]:
res3_dfs=[risui_res3[['routeid_direction', 'type']],passengers_res3,ebitzua_res3,directness_res3,service_areas_res3, complaints_res3]
for i,df in enumerate(res3_dfs):
    if i==0:
        output_res3=df
    else:
        output_res3=output_res3.merge(df, on ='routeid_direction')

In [53]:
res1=ebitzua_res1.merge(complaints_res1, on=['makat','day_period'])
day_periods=res1['day_period'].drop_duplicates().to_frame()
day_periods['tmp'] =1 

res1_dfs=[risui_res12[['makat', 'type']],passengers_res12,directness[['route_desc', 'directness']],service_areas_res12]
for i,df in enumerate(res1_dfs):
    if 'makat' not in df.columns:
        df.rename(columns={'route_desc':'makat'}, inplace=True)
    if i==0:
        output_res1=df
    else:
        output_res1=output_res1.merge(df, on ='makat')

output_res1['tmp'] =1 
ready_tomerge=output_res1.merge(day_periods, on='tmp', how='left')


output_res1=ready_tomerge.merge(res1,on=['makat', 'day_period'], how='left')
output_res1.drop(columns='tmp', inplace=True)

C:\Users\titib\AppData\Local\Temp\ipykernel_8784\1610359753.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'route_desc':'makat'}, inplace=True)


In [54]:
output_res1

,makat,type,passengersnumber,directness,settlements,west,eastwest,injlm,day_period,not_intact_percent,number_of_complaints
0,12035-2-ב,3,7726,0.41,0,0,0,1,3,0.00,1.0
1,12035-2-ב,3,7726,0.41,0,0,0,1,1,0.00,2.0
2,12035-2-ב,3,7726,0.41,0,0,0,1,2,0.00,1.0
3,14023-3-#,3,10446,0.42,0,0,0,1,3,0.00,1.0
4,14023-3-#,3,10446,0.42,0,0,0,1,1,0.00,2.0
...,...,...,...,...,...,...,...,...,...,...,...
1945,10214-3-2,3,54,0.97,0,0,1,0,1,NaN,NaN
1946,10214-3-2,3,54,0.97,0,0,1,0,2,NaN,NaN
1947,10213-3-#,3,249,0.02,0,0,1,1,3,NaN,NaN
1948,10213-3-#,3,249,0.02,0,0,1,1,1,33.33,2.0


In [32]:
res1['day_period'].drop_duplicates().to_frame()

,day_period
0,3
1,1
10,2


In [33]:
# headway_res1_copy=res1[0]['day_period'].drop_duplicates().to_frame()


In [28]:
res1['tmp'] =1 
headway_res1_copy=output_res2[0]['day_period'].drop_duplicates().to_frame()
headway_res1_copy['tmp'] =1 
ready_tomerge=res1.merge(headway_res1_copy, on='tmp', how='left')


KeyError: 0

In [27]:
res1[res1['makat']=='10006-3-#']

,makat,day_period,not_intact_percent,number_of_complaints
0,10006-3-#,3,0.0,1


In [21]:
complaints_res1.sample()

,makat,day_period,number_of_complaints
4778,16059-1-#,2,1


In [ ]:
demographic_table=merge_demographic() 
build_GIS_demographic_data()
bus_lines_jlm=make_jlm_metro_routes()
risui_res12, risui_res3=rishui_resolutions(bus_lines_jlm)
res_12, res3=calculate_socio_econimoc_lines(bus_lines_jlm)

service_areas_res12, service_areas_res3=create_service_areas_resolutions(risui_res12)

passengers_res12, passengers_res3=passengers_resolutions()

ebitzua_res1, ebitzua_res2, ebitzua_res3=ebitzua_resolutions()

complaints_res1,complaints_res2, complaints_res3=complaints_resolutions()

prepare_data_to_directness_analysis()

In [ ]:
# res 2
dfs_inner=[risui_res12, #socio_eco_res12, 
           passengers_res12, directness_res12, service_areas_res12, ebitzua_res2]
dfs_left=[complaints_res2]
res2=merge(dfs_inner, dfs_left, 'makat', 'res2')


In [23]:
passengers_res3.sample()
risui_res3.sample()
service_areas_res3.sample()
ebitzua_res3.sample()
complaints_res2.sample()
directness.sample()

,Unnamed: 0,route_desc,direct_length,trip_km_fact,direct_length_km,directness,routeid,Direction,Alternative,routeid_direction
294,294,12188-2-0,10375.43,11.326,10.375,0.92,12188,2,0,12188-2


In [24]:
directness[['route_desc', 'directness']].sample()

,route_desc,directness
389,13045-2-#,0.65


In [ ]:
directness[['route_desc', 'directness']],complaints_res2,


,makat,number_of_complaints
3564,30066-1-#,2


In [ ]:
directness[['route_desc', 'directness']],complaints_res2,ebitzua_res2,service_areas_res12


,makat,not_intact_percent
5224,27021-2-#,0.0


In [84]:
service_areas_res12[service_areas_res12['west']==1]


,makat,settlements,west,eastwest,injlm
19,13027-1-א,0,1,0,0
21,13027-2-א,0,1,0,0
118,14059-3-#,0,1,0,0
131,11152-1-#,0,1,0,0
132,11152-2-#,0,1,0,0
...,...,...,...,...,...
609,10104-2-#,0,1,0,0
620,10150-1-#,0,1,0,0
621,10150-2-#,0,1,0,0
660,11185-2-2,0,1,0,0


In [ ]:
complaints_res2,ebitzua_res2,service_areas_res12,risui_res12[['makat', 'type']],passengers_res12


,makat,type
1219,12065-1-ד,3


In [ ]:
risui_res12[['makat', 'type']],passengers_res12,ebitzua_res2,directness[['route_desc', 'directness']],service_areas_res12

,makat,passengersnumber
6222,42051-2-#,89


In [ ]:
res2_dfs=[risui_res12[['makat', 'type']],passengers_res12,ebitzua_res2,directness[['route_desc', 'directness']],service_areas_res12]
for i,df in enumerate(res2_dfs):
    if 'makat' not in df.columns:
        df.rename(columns={'route_desc':'makat'}, inplace=True)
    if i==0:
        output_res2=df
    else:
        output_res2=output_res2.merge(df, on ='makat')

In [55]:
passengers_res3

,routeid_direction,passengersnumber
0,10001-3,8580
1,10003-1,441
2,10006-1,45
3,10006-3,730
4,10007-1,364
...,...,...
5381,98005-2,3885
5382,99002-1,10204
5383,99002-2,10124
5384,99005-1,2799


In [57]:
res3_dfs=[risui_res3[['routeid_direction', 'type']],passengers_res3,ebitzua_res3,directness_res3,service_areas_res3]
for i,df in enumerate(res3_dfs):
    if i==0:
        output_res3=df
    else:
        output_res3=output_res3.merge(df, on ='routeid_direction')

In [66]:
output_res3.sample(10)

,routeid_direction,type,passengersnumber,not_intact_percent,directness,settlements,west,eastwest,injlm
201,11793-1,3,102,20.00,0.3000,0,0,0.0,1
404,15019-2,3,26617,0.00,0.8825,0,0,0.0,1
445,17055-2,3,276,0.00,0.4200,0,0,0.0,1
352,13270-2,2,6,0.00,0.5600,0,0,0.0,0
197,11787-1,3,339,0.00,0.7800,0,0,0.0,1
170,11225-2,1,3327,2.22,0.6700,0,0,0.0,0
272,12242-1,1,100,0.00,0.7100,0,0,0.0,0
78,10780-2,3,195,0.00,0.6200,0,0,0.0,1
177,11264-1,2,35,0.00,0.4300,0,0,0.0,0
437,16222-1,1,43,0.00,0.7100,0,0,0.0,0


In [9]:
def merge(dfs_inner, dfs_left, merging_word, res_num):
    res12=pd.DataFrame()
    throw_columns = ['routeid', 'complaints']
    change_columns = {'route_desc': 'makat'}
    for int, df in enumerate(dfs_inner):
        df = df.drop(columns=[col for col in df if col in throw_columns]) #throw unnecessary columns
        df.rename(columns=change_columns, inplace=True)

        if int ==0:
            res12=df.copy()
        else:
            res12=res12.merge(df, on=merging_word, how='inner')

    for int, df in enumerate(dfs_left):
        df = df.drop(columns=[col for col in df if col in throw_columns]) #throw unnecessary columns
        if type(merging_word)=='list':
            if int==0:
                res12=res12.merge(df, on=['makat'], how='left')
            else:
                res12=res12.merge(df, on=merging_word, how='left')
        else:
            res12=res12.merge(df, on=merging_word, how='left')

    # fix merging
    res12['number_of_complaints'].fillna(0, inplace=True)
    res12.loc[res12['settlements']==1, 'west'] = 0
    res12['passengersnumber_thousands']=res12['passengersnumber']/1000
    
    # res12.to_csv(f'saved_steps/{res_num}.csv')
    return res12

In [10]:
# res 2
dfs_inner=[risui_res12, #socio_eco_res12, 
           passengers_res12, directness_res12, service_areas_res12, ebitzua_res2]
dfs_left=[complaints_res2]
res2=merge(dfs_inner, dfs_left, 'makat', 'res2')

# res 3
dfs_inner=[risui_res3, #socio_eco_res3, 
           passengers_res3, directness_res3,service_areas_res3, ebitzua_res3]
dfs_left=[complaints_res3]
res3=merge(dfs_inner, dfs_left, 'routeid_direction', 'res3')

In [86]:
service_areas_res3=service_areas_res12.drop(columns='makat').groupby('routeid_direction', as_index=False).mean()


KeyError: 'routeid_direction'

In [16]:
res2.columns

Index(['makat', 'routeid_direction', 'type', 'route_length', 'eastwest_x',
       'passengersnumber', 'directness', 'settlements', 'west', 'eastwest_y',
       'injlm', 'not_intact_percent', 'number_of_complaints',
       'passengersnumber_thousands'],
      dtype='object')

In [ ]:
# res1
def merge_res1(dfs_inne`r, dfs_left, res_num):
    res12=pd.DataFrame()
    throw_columns = ['routeid', 'complaints']
    change_columns = {'route_desc': 'makat'}
    for int, df in enumerate(dfs_inner):
        # df = df.drop(columns=[col for col in df if col in throw_columns]) #throw unnecessary columns
        df.rename(columns=change_columns, inplace=True)
        if int ==0:
            res12=df.copy()
        else:
            res12=res12.merge(df, on='makat', how='inner')

    res1_start=res12.drop_duplicates()
    res1_start['tmp'] =1 
    headway_res1_copy=dfs_left[0]['day_period'].drop_duplicates().to_frame()
    headway_res1_copy['tmp'] =1 
    ready_tomerge=res1_start.merge(headway_res1_copy, on='tmp', how='left')

    for int, df in enumerate(dfs_left):

        ready_tomerge=ready_tomerge.merge(df, on=['makat', 'day_period'], how='left')

    ready_tomerge['number_of_complaints'].fillna(0, inplace=True)
    ready_tomerge['headway'].fillna(0, inplace=True)
    ready_tomerge['not_intact_percent'].fillna(0, inplace=True)

    # ready_tomerge.dropna(subset=['not_intact_percent'], inplace=True)

    ready_tomerge.loc[ready_tomerge['settlements']==1, 'west'] = 0
    ready_tomerge['passengersnumber_thousands']=ready_tomerge['passengersnumber']/1000
    # ready_tomerge.to_csv('saved_steps/{}.csv'.format(res_num))

    return ready_tomerge.drop_duplicates()

dfs_inner=[rishui_res12, #socio_eco_res12, 
           passengers_res12, directness_res12, east_res12, west_res12, injlm_res12]
dfs_left=[headway_res1, ebitzua_res1, complaints_res1]
res1=merge_res1(dfs_inner, dfs_left, 'res1')


KeyError: 'day_period'

In [ ]:
demographic_table=merge_demographic() 
build_GIS_demographic_data()
bus_lines_jlm=make_jlm_metro_routes()
risui_res12, risui_res3=rishui_resolutions(bus_lines_jlm)
res_12, res3=calculate_socio_econimoc_lines(bus_lines_jlm)

service_areas_res12, service_areas_res3=create_service_areas_resolutions(risui_res12)

passengers_res12, passengers_res3=passengers_resolutions()

ebitzua_res1, ebitzua_res2, ebitzua_res3=ebitzua_resolutions()

complaints_res1,complaints_res2, complaints_res3=complaints_resolutions()

prepare_data_to_directness_analysis()